# YOLOv5 Face Mask Detection 🧪
This notebook walks through loading a trained YOLOv5 model and testing it on images.

In [ ]:
import os
import zipfile
import numpy as np
from tkinter import Tk
from tkinter.filedialog import askopenfilename
from kaggle.api.kaggle_api_extended import KaggleApi
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# --- Step 1: Upload kaggle.json via file picker ---
def upload_kaggle_json():
    print("Pilih file kaggle.json untuk autentikasi Kaggle API...")
    Tk().withdraw()  # sembunyikan jendela utama tkinter
    file_path = askopenfilename(title="Pilih file kaggle.json", filetypes=[("JSON files", "*.json")])
    if not file_path:
        print("File kaggle.json tidak dipilih. Program dihentikan.")
        exit(1)
    print(f"File kaggle.json dipilih: {file_path}")
    # Set config dir ke folder file json
    os.environ['KAGGLE_CONFIG_DIR'] = os.path.dirname(file_path)
    return file_path

In [ ]:
# --- Step 2: Download dataset dari Kaggle ---
def download_dataset():
    api = KaggleApi()
    api.authenticate()
    print("Download dataset Face Mask Detection dari Kaggle...")
    api.dataset_download_files('andrewmvd/face-mask-detection', path='datasets', unzip=True)
    print("Download selesai!")

In [ ]:
# --- Step 3: Prepare data generators (preprocessing & augmentation) ---
def create_data_generators(img_size=(128,128), batch_size=32):
    base_dir = os.path.join('datasets', 'images')

    # Dataset sudah terstruktur folder: train/validation belum ada, jadi kita pakai ImageDataGenerator dengan validation_split
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        rotation_range=15,
        shear_range=0.1,
        fill_mode='nearest'
    )

    train_gen = datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )

    val_gen = datagen.flow_from_directory(
        base_dir,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )

    return train_gen, val_gen

In [ ]:
# --- Step 4: Build CNN model ---
def build_model(input_shape=(128,128,3), num_classes=3):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# --- Step 5: Plot training history ---
def plot_history(history):
    plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.legend()
    plt.title('Loss')

    plt.show()

In [ ]:
# --- Step 6: Evaluate model performance on validation data ---
def evaluate_model(model, val_gen):
    val_gen.reset()
    preds = model.predict(val_gen, verbose=1)
    y_pred = np.argmax(preds, axis=1)
    y_true = val_gen.classes
    class_labels = list(val_gen.class_indices.keys())

    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_labels))

    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

In [ ]:
# --- Main pipeline ---
def main():
    # Upload file kaggle.json
    upload_kaggle_json()

    # Download dataset dan ekstrak
    download_dataset()

    # Prepare data generators
    train_gen, val_gen = create_data_generators()

    # Build model
    model = build_model()

    # Train model
    print("Mulai training...")
    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10
    )

    # Plot training history
    plot_history(history)

    # Evaluate model
    evaluate_model(model, val_gen)

if __name__ == '__main__':
    main()